## 1. Connect Database

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path("D:/code/text-to-sql-agent")))
from db.db_connection import get_db_connection

conn = get_db_connection()
cursor = conn.cursor()

## 2. Discover Database Structure

In [ ]:
# 2.1 List all user tables

cursor.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
""")

tables = [row[0] for row in cursor.fetchall()]
tables


['aisles',
 'departments',
 'order_products_prior',
 'order_products_train',
 'orders',
 'products']

In [ ]:
# 2.2 Extract columns for all tables

cursor.execute("""
SELECT
    table_name,
    column_name,
    data_type,
    is_nullable,
    ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public'
ORDER BY table_name, ordinal_position;
""")

columns_raw = cursor.fetchall()
columns_raw[:10]  # sanity check

[('aisles', 'aisle_id', 'bigint', 'NO', 1),
 ('aisles', 'aisle', 'text', 'YES', 2),
 ('departments', 'department_id', 'bigint', 'NO', 1),
 ('departments', 'department', 'text', 'YES', 2),
 ('order_products_prior', 'order_id', 'bigint', 'YES', 1),
 ('order_products_prior', 'product_id', 'bigint', 'YES', 2),
 ('order_products_prior', 'add_to_cart_order', 'bigint', 'YES', 3),
 ('order_products_prior', 'reordered', 'bigint', 'YES', 4),
 ('order_products_train', 'order_id', 'bigint', 'YES', 1),
 ('order_products_train', 'product_id', 'bigint', 'YES', 2)]

In [ ]:
# 2.3 Extract primary keys

cursor.execute("""
SELECT
    tc.table_name,
    kcu.column_name
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu
  ON tc.constraint_name = kcu.constraint_name
WHERE tc.constraint_type = 'PRIMARY KEY'
  AND tc.table_schema = 'public'
ORDER BY tc.table_name, kcu.ordinal_position;
""")

primary_keys = cursor.fetchall()
primary_keys

[('aisles', 'aisle_id'),
 ('departments', 'department_id'),
 ('orders', 'order_id'),
 ('products', 'product_id')]

In [ ]:
# 2.4 Extract foreign keys

cursor.execute("""
SELECT
    tc.table_name,
    kcu.column_name,
    ccu.table_name AS foreign_table,
    ccu.column_name AS foreign_column
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu
  ON tc.constraint_name = kcu.constraint_name
JOIN information_schema.constraint_column_usage ccu
  ON ccu.constraint_name = tc.constraint_name
WHERE tc.constraint_type = 'FOREIGN KEY'
  AND tc.table_schema = 'public'
ORDER BY tc.table_name;
""")

foreign_keys = cursor.fetchall()
foreign_keys

[('order_products_prior', 'order_id', 'orders', 'order_id'),
 ('order_products_prior', 'product_id', 'products', 'product_id'),
 ('order_products_train', 'order_id', 'orders', 'order_id'),
 ('order_products_train', 'product_id', 'products', 'product_id'),
 ('products', 'aisle_id', 'aisles', 'aisle_id'),
 ('products', 'department_id', 'departments', 'department_id')]

## 3. Normalize schema into a dict

In [11]:
from collections import defaultdict

# Initialize schema container
schema_dict = {
    "tables": {}
}

# Populate tables and columns
for table, column, dtype, nullable, position in columns_raw:
    if table not in schema_dict["tables"]:
        schema_dict["tables"][table] = {
            "columns": {},
            "primary_key": [],
            "foreign_keys": []
        }

    schema_dict["tables"][table]["columns"][column] = {
        "data_type": dtype,
        "nullable": nullable == "YES",
        "ordinal_position": position
    }

# Attach primary keys
for table, column in primary_keys:
    schema_dict["tables"][table]["primary_key"].append(column)

# Attach foreign key relationships
for table, column, foreign_table, foreign_column in foreign_keys:
    schema_dict["tables"][table]["foreign_keys"].append({
        "column": column,
        "references": {
            "table": foreign_table,
            "column": foreign_column
        }
    })

In [ ]:
# Sanity Check

from IPython.display import display

display(schema_dict.keys())
display(schema_dict["tables"].keys())
display(schema_dict["tables"]["orders"])

dict_keys(['tables'])

dict_keys(['aisles', 'departments', 'order_products_prior', 'order_products_train', 'orders', 'products'])

{'columns': {'order_id': {'data_type': 'bigint',
   'nullable': False,
   'ordinal_position': 1},
  'user_id': {'data_type': 'bigint', 'nullable': True, 'ordinal_position': 2},
  'eval_set': {'data_type': 'text', 'nullable': True, 'ordinal_position': 3},
  'order_number': {'data_type': 'bigint',
   'nullable': True,
   'ordinal_position': 4},
  'order_dow': {'data_type': 'bigint',
   'nullable': True,
   'ordinal_position': 5},
  'order_hour_of_day': {'data_type': 'bigint',
   'nullable': True,
   'ordinal_position': 6},
  'days_since_prior_order': {'data_type': 'double precision',
   'nullable': True,
   'ordinal_position': 7}},
 'primary_key': ['order_id'],
 'foreign_keys': []}

## 4. Semantic Enrichment (LLM-critical)

In [15]:
# 1️Table-level descriptions (manual, explicit)
table_descriptions = {
    "orders": "One row per customer order, including timing and sequence information",
    "order_products_prior": "Products included in prior (historical) orders",
    "order_products_train": "Products included in training orders (used for prediction)",
    "products": "Product master table with category information",
    "aisles": "Product aisle categories (fine-grained grouping)",
    "departments": "Product department categories (coarse-grained grouping)"
}

for table, desc in table_descriptions.items():
    schema_dict["tables"][table]["description"] = desc


# 2️ Column-level semantic meaning (business semantics)
# This is intentionally NOT autogenerated.
column_descriptions = {
    "orders": {
        "order_id": "Unique identifier for an order",
        "user_id": "Customer who placed the order",
        "order_number": "Sequence number of the order for a user",
        "order_dow": "Day of week when the order was placed (0=Saturday, 6=Friday)",
        "order_hour_of_day": "Hour of day when the order was placed (0–23)",
        "days_since_prior_order": "Days since the user's previous order; NULL for first order"
    },
    "order_products_prior": {
        "order_id": "Order identifier",
        "product_id": "Product included in the order",
        "add_to_cart_order": "Sequence in which the product was added to cart",
        "reordered": "1 if the product was ordered by the user previously, else 0"
    },
    "order_products_train": {
        "order_id": "Order identifier",
        "product_id": "Product included in the order",
        "add_to_cart_order": "Sequence in which the product was added to cart",
        "reordered": "1 if the product was ordered by the user previously, else 0"
    },
    "products": {
        "product_id": "Unique product identifier",
        "product_name": "Human-readable product name",
        "aisle_id": "Aisle category of the product",
        "department_id": "Department category of the product"
    },
    "aisles": {
        "aisle_id": "Unique aisle identifier",
        "aisle": "Name of the aisle"
    },
    "departments": {
        "department_id": "Unique department identifier",
        "department": "Name of the department"
    }
}

for table, cols in column_descriptions.items():
    for col, meaning in cols.items():
        schema_dict["tables"][table]["columns"][col]["description"] = meaning


# 3️ Global semantic hints (reduce logical SQL errors)
schema_dict["hints"] = [
    "orders.days_since_prior_order is NULL for a user's first order",
    "reordered = 1 indicates the user has purchased the product before",
    "order_products_prior contains historical orders; order_products_train is used for model training",
    "Products must be joined via order_products_* tables to access order-level information"
]

## 5. Define common join paths (anti-hallucination)

In [16]:
schema_dict["common_joins"] = [
    {
        "description": "Products in historical orders",
        "tables": ["orders", "order_products_prior", "products"],
        "joins": [
            {
                "from": "orders.order_id",
                "to": "order_products_prior.order_id",
                "type": "INNER"
            },
            {
                "from": "order_products_prior.product_id",
                "to": "products.product_id",
                "type": "INNER"
            }
        ]
    },
    {
        "description": "Products in training orders",
        "tables": ["orders", "order_products_train", "products"],
        "joins": [
            {
                "from": "orders.order_id",
                "to": "order_products_train.order_id",
                "type": "INNER"
            },
            {
                "from": "order_products_train.product_id",
                "to": "products.product_id",
                "type": "INNER"
            }
        ]
    },
    {
        "description": "Product with aisle information",
        "tables": ["products", "aisles"],
        "joins": [
            {
                "from": "products.aisle_id",
                "to": "aisles.aisle_id",
                "type": "INNER"
            }
        ]
    },
    {
        "description": "Product with department information",
        "tables": ["products", "departments"],
        "joins": [
            {
                "from": "products.department_id",
                "to": "departments.department_id",
                "type": "INNER"
            }
        ]
    },
    {
        "description": "Complete product hierarchy (product → aisle → department)",
        "tables": ["products", "aisles", "departments"],
        "joins": [
            {
                "from": "products.aisle_id",
                "to": "aisles.aisle_id",
                "type": "INNER"
            },
            {
                "from": "products.department_id",
                "to": "departments.department_id",
                "type": "INNER"
            }
        ],
        "note": "Aisles and departments are independent categorizations of products"
    }
]

# 6. Export schema summary to YAML

In [19]:
import yaml
from pathlib import Path

output_path = Path("../src/schema/schema_summary.yaml")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w", encoding="utf-8") as f:
    yaml.dump(schema_dict, f, sort_keys=False, allow_unicode=True)

print(f"Schema summary written to: {output_path.resolve()}")

Schema summary written to: D:\code\text-to-sql-agent\src\schema\schema_summary.yaml
